<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_3)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to spawn a process**

In [4]:
import multiprocessing

def foo(i):
  print ('called function in process: %s' %i)
  return

if __name__ == '__main__':
  Process_jobs = []
  for i in range(5):
    p = multiprocessing.Process(target=foo, args=(i,))
    Process_jobs.append(p)
    p.start()
    p.join()


called function in process: 0
called function in process: 1
called function in process: 2
called function in process: 3
called function in process: 4


**How to name a process**

Процесс обращения к имени процесса схож с тем, что был во второй главе во время обращения к имени потока.

In [6]:
import multiprocessing
import time

def foo():
  name = multiprocessing.current_process().name
  print("Starting %s \n" %name)
  time.sleep(3)
  print("Exiting %s \n %name")

if __name__ == '__main__':
  process_with_name = multiprocessing.Process(name='foo_process', target=foo)
  process_with_name.daemon = True
  process_with_default_name = multiprocessing.Process(target=foo)
  process_with_name.start()
  process_with_default_name.start()

Starting foo_process 

Starting Process-19 



**How to run a process in the background**

In [12]:
import multiprocessing
import time

def foo():
 name = multiprocessing.current_process().name
 print ("Starting %s \n" %name)
 time.sleep(3)
 print ("Exiting %s \n" %name)

if __name__ == '__main__':
  background_process = multiprocessing.Process(name='background_process', target=foo)
  background_process.daemon = True

  NO_background_process = multiprocessing.Process(name='NO_background_process', target=foo)
  NO_background_process.daemon = False

  background_process.start()
  NO_background_process.start()
  
  background_process.join()
  NO_background_process.join()



Starting background_process 

Starting NO_background_process 

Exiting background_process 

Exiting NO_background_process 



**How to kill a process**